In [1]:
# Method to query data from google spreadsheet as a pandas dataframe
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient import discovery
import pandas as pd
import numpy as np
import gspread
pd.options.mode.chained_assignment = None  # default='warn'

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('stock-scanner-project-75f48f4873c1.json',scope)
service = discovery.build('sheets', 'v4', credentials=credentials)
gs = service.spreadsheets()
spreadsheet_id  = "1yGCigPgAQUggnkg2qlOhHxHwEnH2fjfcheBgJsNi-5Q"
# sheet = gs.values().get(spreadsheetId = spreadsheet_id, range = range_, valueRenderOption = 'FORMULA').execute()
# df_values = pd.DataFrame(sheet.get('values')[0:],columns=sheet.get('values')[0])
# df_values = df_values[1:].reset_index(drop = True)

In [2]:
# reads sheet 'NSE Holidays' from spreadsheet 'Stock_Scanner_Python'
nse_holidays_sheet = gs.values().get(spreadsheetId = spreadsheet_id, 
                                     range = "NSE Holidays", 
                                     valueRenderOption = 'FORMULA',
                                     dateTimeRenderOption = 'FORMATTED_STRING').execute()
df_nse_holidays = pd.DataFrame(nse_holidays_sheet.get('values')[0:],columns=nse_holidays_sheet.get('values')[0])
df_nse_holidays = df_nse_holidays[1:].reset_index(drop = True)

In [3]:
import datetime as dt
nse_holidays = [dt.datetime.strptime(date, "%d-%b-%Y").date() for date in df_nse_holidays['Date']]

In [4]:
from datetime import date
from nsepy import get_history

In [5]:
# function to get previous trading day
from datetime import date, timedelta
def prev_weekday(adate):
    adate -= timedelta(days=1)
    while ((adate.weekday() > 4) & (adate not in nse_holidays)):
        # Mon-Fri are 0-4 #also add holidays to this
        adate -= timedelta(days=1)
    return adate

prev_weekday(date.today())

datetime.date(2021, 11, 17)

In [6]:
split_dates = pd.read_csv("stock_split_dates.csv")
split_dates['Split_Day'] = pd.to_datetime(split_dates['Split_Day'])

In [7]:
# split_dates

In [8]:
def max_prev_weekday_split_date(prev_weekday,stock_name):
    stock_name = stock_name.upper()
    try:
        split_date = dt.datetime.strptime(split_dates.loc[split_dates['Name']==stock_name,
                                     'Split_Day'].values[0].astype(str).split(sep = 'T')[0], 
                     "%Y-%m-%d")
        split_date = dt.datetime.date(split_date)
        prev_weekday = max(prev_weekday,split_date)
        return(prev_weekday)
        print(split_date)
        print(prev_weekday)
    except Exception as e:
#         print(e)
        return(prev_weekday)
#         return(prev_weekday)

In [9]:
def trend_(dataframe,days = 20):
    dataframe['Trend+'] = (dataframe['Smooth_+DX']>dataframe['Smooth_-DX']).astype(int)
    dataframe['Trend-'] = (dataframe['Smooth_-DX']>dataframe['Smooth_+DX']).astype(int)
    if(np.sum(dataframe['Trend+'])==len(dataframe['Trend+'])):
        return('uptrend')
    elif(np.sum(dataframe['Trend-'])==len(dataframe['Trend-'])):
        return('downtrend')
    else:
        return('')

In [10]:
def get_stock_data(stock):
    stock = stock.lower()
    
    if(stock == 'banknifty'):
        stock = 'NIFTY Bank'
        a = get_history(symbol=stock, 
                        start=prev_weekday(date.today())-timedelta(weeks=52),
                        end=date.today(),
                        index = True)
        a['Prev Close'] = a['Close'].shift(periods=1)
        a['Stock Name'] = stock

    elif(stock == 'nifty'):
        stock = 'NIFTY 50'
        a = get_history(symbol=stock, 
                        start=prev_weekday(date.today())-timedelta(weeks=52),
                        end=date.today(),
                        index = True)
        a['Prev Close'] = a['Close'].shift(periods=1)
        a['Stock Name'] = stock

    else:
        a = get_history(symbol=stock, 
                        start=max_prev_weekday_split_date(prev_weekday(date.today())-timedelta(weeks=52),
                                                          stock.upper()),
                        end=date.today())
        if(a.shape[0] == 0):
            a = get_history(symbol=stock.upper(), 
                            start=max_prev_weekday_split_date(prev_weekday(date.today())-timedelta(weeks=52),stock.upper()),
                            end=date.today())
        a['Stock Name'] =stock.upper()
    
    a = a[['Stock Name','Open','High','Low','Close','Prev Close','Volume']]
    a['high_minus_low'] = a['High'] - a['Low']
    a['high_minus_prev_close'] = a['High'] - a['Prev Close']
    a['low_minus_prev_close'] = a['Low'] - a['Prev Close']
    a['True_Range'] = a[['high_minus_low','high_minus_prev_close','low_minus_prev_close']].max(axis=1)
    a['ATR'] = a['True_Range'].rolling(window = 14, min_periods= 14).mean()
    a['Prev High'] = a['High'].shift(1)
    a['High_minus_Prev_High'] = a['High'] - a['Prev High']
    a['Prev Low'] = a['Low'].shift(1)
    a['Prev_Low_minus_Low'] = a['Prev Low'] - a['Low']
    a['+DX'] = a.apply(lambda x: x['High_minus_Prev_High'] if((x['High_minus_Prev_High']>x['Prev_Low_minus_Low']) & 
                                                              (x['High_minus_Prev_High']>0)) else 0, axis = 1)
    a['-DX'] = a.apply(lambda x: x['Prev_Low_minus_Low'] if((x['Prev_Low_minus_Low']>x['High_minus_Prev_High']) & 
                                                            (x['Prev_Low_minus_Low']>0)) else 0, axis = 1)
    a['Smooth_+DX'] = a['+DX'].rolling(window = 14, min_periods = 14).mean()
    a['Smooth_-DX'] = a['-DX'].rolling(window = 14, min_periods = 14).mean()
    a['+DMI'] = a['Smooth_+DX']/a['ATR']
    a['-DMI'] = a['Smooth_-DX']/a['ATR']
    a['DX'] = np.abs(a['+DMI']-a['-DMI'])*100.0/(a['+DMI']+a['-DMI'])
    a['ADX'] = a['DX'].rolling(window = 14, min_periods = 14).mean()
    trend_stock = trend_(a.tail(20))
    b = a.iloc[-1]
    b['Trend'] = trend_stock
    b['20_D_Avg_Vol'] = np.average(a.tail(20)['Volume'])
    b['10_D_Avg_Vol'] = np.average(a.tail(10)['Volume'])
    b['5_D_Avg_Vol'] = np.average(a.tail(5)['Volume'])
    
#     b['50_D_Avg_P'] = np.average(a.tail(50)['Price'])
#     b['20_D_Avg_P'] = np.average(a.tail(20)['Price'])
#     b['5_D_Avg_P'] = np.average(a.tail(5)['Price'])

    b['200_D_Avg_Close'] = np.average(a.tail(200)['Close'])
    b['50_D_Avg_Close'] = np.average(a.tail(50)['Close'])
    b['20_D_Avg_Close'] = np.average(a.tail(20)['Close'])
    b['5_D_Avg_Close'] = np.average(a.tail(5)['Close'])
    
    b['Prv_200_D_Avg_Close'] = np.average(a.tail(201)[1:200]['Close'])
    b['Prv_50_D_Avg_Close'] = np.average(a.tail(51)[1:50]['Close'])
    b['Prv_20_D_Avg_Close'] = np.average(a.tail(21)[1:20]['Close'])

    
    b['120_D_Max_Close'] = np.max(a.tail(120)['Close'])
    b['20_D_Max_Close'] = np.max(a.tail(20)['Close'])
    b['5_D_Max_Close'] = np.average(a.tail(5)['Close'])

    b['Yr_H_Price'] = np.max(a['High'])
    b['Yr_L_Price'] = np.min(a['Low'])
    b['Yr_H_Vol'] = np.max(a['Volume'])
    b['Yr_L_Vol'] = np.min(a['Volume'])
    
    b['Prv_Yr_H_Price'] = np.max(a[:-1]['High'])
    b['Prv_Yr_L_Price'] = np.min(a[:-1]['Low'])
    b['Prv_Yr_H_Vol'] = np.max(a[:-1]['Volume'])
    b['Prv_Yr_L_Vol'] = np.min(a[:-1]['Volume'])
    
    b['Prv_2_Close'] = a.iloc[-3]['Close']
    b['Prv_3_Close'] = a.iloc[-4]['Close']
    
    b['Prv_1_Open'] = a.iloc[-2]['Open']
    b['Prv_2_Open'] = a.iloc[-3]['Open']
    b['Prv_3_Open'] = a.iloc[-4]['Open']

    b['Prv_1_Low'] = a.iloc[-2]['Low']
    b['Prv_2_Low'] = a.iloc[-3]['Low']
    b['Prv_3_Low'] = a.iloc[-4]['Low']
    b['Prv_4_Low'] = a.iloc[-5]['Low']
    b['Prv_5_Low'] = a.iloc[-6]['Low']
    b['Prv_6_Low'] = a.iloc[-7]['Low']
    b['Prv_7_Low'] = a.iloc[-8]['Low']
    b['Prv_8_Low'] = a.iloc[-9]['Low']
    b['10_D_Min_Low'] = np.min(a.tail(10)['Low'])
    b['50_D_Min_Low'] = np.min(a.tail(50)['Low'])

    b['Prv_1_High'] = a.iloc[-2]['High']
    b['Prv_2_High'] = a.iloc[-3]['High']
    b['Prv_3_High'] = a.iloc[-4]['High']
    b['Prv_4_High'] = a.iloc[-5]['High']
    b['Prv_5_High'] = a.iloc[-6]['High']
    b['Prv_6_High'] = a.iloc[-7]['High']
    b['Prv_7_High'] = a.iloc[-8]['High']
    b['Prv_8_High'] = a.iloc[-9]['High']
    b['10_D_Max_High'] = np.max(a.tail(10)['High'])
    b['50_D_Max_High'] = np.max(a.tail(50)['High'])
    
    b['100_D_Avg_Range'] = np.mean(a.tail(100)['High']-a.tail(100)['Low'])
    b['Prev_Volume'] = a.iloc[-2]['Volume']
    
    b = pd.DataFrame(b.T.reset_index().T.values[1:],columns = b.T.reset_index().T.values[0])
    return b

## Nifty F&O

In [11]:
date.today()

datetime.date(2021, 11, 18)

In [12]:
## proceed only if this code returns latest data
stock = "itc"
get_history(symbol=stock, 
            start=max_prev_weekday_split_date(prev_weekday(date.today())-timedelta(weeks=2),
                                              stock.upper()),
            end=date.today())

Symbol Series  Prev Close    Open    High     Low    Last   Close  \
Date                                                                           
2021-11-03    ITC     EQ      225.60  226.00  226.85  221.70  222.75  222.45   
2021-11-04    ITC     EQ      222.45  224.85  227.30  224.00  227.15  226.55   
2021-11-08    ITC     EQ      226.55  229.70  230.70  225.70  229.65  229.80   
2021-11-09    ITC     EQ      229.80  229.85  230.90  227.85  228.85  228.75   
2021-11-10    ITC     EQ      228.75  227.45  232.85  227.20  230.75  230.85   
2021-11-11    ITC     EQ      230.85  230.40  231.50  228.55  230.20  230.00   
2021-11-12    ITC     EQ      230.00  231.20  233.95  229.15  232.85  232.90   
2021-11-15    ITC     EQ      232.90  233.95  239.00  233.10  238.15  238.10   
2021-11-16    ITC     EQ      238.10  239.05  239.65  235.00  235.25  235.45   
2021-11-17    ITC     EQ      235.45  236.00  245.25  235.65  239.30  239.50   

              VWAP    Volume      Turnover  Trades  Deliverable Volume  \
Date                                                                     
2021-11-03  223.82  17843327  3.993623e+14  119495            11487403   
2021-11-04  225.88   7024796  1.586768e+14  106228             4126838   
2021-11-08  228.22  20270558  4.626148e+14  168607             9872907   
2021-11-09  229.11  11520817  2.639533e+14  131594             5224823   
2021-11-10  230.52  13431108  3.096183e+14  110805             4693940   
2021-11-11  229.65   8134962  1.868180e+14   96104             3356619   
2021-11-12  231.88  11855531  2.749079e+14   99013             4459244   
2021-11-15  236.77  38755267  9.176154e+14  176533            21205338   
2021-11-16  237.36  14675969  3.483463e+14   97861             7990381   
2021-11-17  240.97  46697822  1.125290e+15  246949            15742715   

            %Deliverble  
Date                     
2021-11-03       0.6438  
2021-11-04       0.5875  
2021-11-08       0.4871  
2021-11-09       0.4535  
2021-11-10       0.3495  
2021-11-11       0.4126  
2021-11-12       0.3761  
2021-11-15       0.5472  
2021-11-16       0.5445  
2021-11-17       0.3371

In [13]:
nifty_f_n_o = pd.read_excel("stock_names.xlsx",header=0,sheet_name="NSE F&O")
# nifty_500 = pd.read_excel("stock_names.xlsx",header=0,sheet_name="NSE 500")

In [14]:
stock_data_nse_f_n_o = pd.DataFrame(columns = ['Stock Name', 'Open','High','Low','Close','Prev Close',
                                     'Yr_H_Price','Yr_L_Price','Trend',
                                     '200_D_Avg_Close','Prv_200_D_Avg_Close','120_D_Max_Close','50_D_Avg_Close',
                                     'Prv_50_D_Avg_Close','Prv_20_D_Avg_Close','20_D_Max_Close','20_D_Avg_Close',
                                     '5_D_Max_Close','5_D_Avg_Close','Prv_3_Close','Prv_2_Close',
                                     '50_D_Max_High','10_D_Max_High',
                                     'Prv_7_High','Prv_6_High','Prv_5_High','Prv_4_High','Prv_3_High','Prv_2_High','Prv_1_High',
                                     '50_D_Min_Low','10_D_Min_Low',
                                     'Prv_7_Low','Prv_6_Low','Prv_5_Low','Prv_4_Low','Prv_3_Low','Prv_2_Low','Prv_1_Low',
                                     '100_D_Avg_Range',
                                     'Yr_H_Vol','Yr_L_Vol',
                                     '20_D_Avg_Vol','10_D_Avg_Vol','5_D_Avg_Vol',
                                     'Volume','Prev_Volume',
                                     'Prv_1_Open','Prv_2_Open','Prv_3_Open',
                                     'Prv_Yr_H_Price','Prv_Yr_L_Price','Prv_Yr_H_Vol','Prv_Yr_L_Vol',
                                     'Prv_8_High','Prv_8_Low'])

# stock_names = nifty_500['NSE 500 Stocks']
stock_names = nifty_f_n_o['NSE Futures 150 stocks']

from tqdm import tqdm_notebook as tqdm
for stock in tqdm(stock_names):
    temp = get_stock_data(stock)
    temp = temp[['Stock Name', 'Open','High','Low','Close','Prev Close',
                                     'Yr_H_Price','Yr_L_Price','Trend',
                                     '200_D_Avg_Close','Prv_200_D_Avg_Close','120_D_Max_Close','50_D_Avg_Close',
                                     'Prv_50_D_Avg_Close','Prv_20_D_Avg_Close','20_D_Max_Close','20_D_Avg_Close',
                                     '5_D_Max_Close','5_D_Avg_Close','Prv_3_Close','Prv_2_Close',
                                     '50_D_Max_High','10_D_Max_High',
                                     'Prv_7_High','Prv_6_High','Prv_5_High','Prv_4_High','Prv_3_High','Prv_2_High','Prv_1_High',
                                     '50_D_Min_Low','10_D_Min_Low',
                                     'Prv_7_Low','Prv_6_Low','Prv_5_Low','Prv_4_Low','Prv_3_Low','Prv_2_Low','Prv_1_Low',
                                     '100_D_Avg_Range',
                                     'Yr_H_Vol','Yr_L_Vol',
                                     '20_D_Avg_Vol','10_D_Avg_Vol','5_D_Avg_Vol',
                                     'Volume','Prev_Volume',
                                     'Prv_1_Open','Prv_2_Open','Prv_3_Open',
                                     'Prv_Yr_H_Price','Prv_Yr_L_Price','Prv_Yr_H_Vol','Prv_Yr_L_Vol',
                                     'Prv_8_High','Prv_8_Low']]
    stock_data_nse_f_n_o=stock_data_nse_f_n_o.append(temp, ignore_index = True)

<ipython-input-14-cb05d1c43d4d>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for stock in tqdm(stock_names):


  0%|          | 0/144 [00:00<?, ?it/s]

In [15]:
stock_data_nse_f_n_o.shape

(144, 56)

In [16]:
# stock_names

In [17]:
nifty_f_n_o_stocks_data = pd.merge(stock_data_nse_f_n_o, nifty_f_n_o, how='left', left_on=['Stock Name'], 
                                   right_on = [nifty_f_n_o.columns[0]])
# # to convert dataframe to list for uploading to google sheet
nifty_f_n_o_stocks_data = nifty_f_n_o_stocks_data[['Stock Name', 'Sector', 'Open','High','Low','Close','Prev Close',
                                     'Yr_H_Price','Yr_L_Price','Trend',
                                     '200_D_Avg_Close','Prv_200_D_Avg_Close','120_D_Max_Close','50_D_Avg_Close',
                                     'Prv_50_D_Avg_Close','Prv_20_D_Avg_Close','20_D_Max_Close','20_D_Avg_Close',
                                     '5_D_Max_Close','5_D_Avg_Close','Prv_3_Close','Prv_2_Close',
                                     '50_D_Max_High','10_D_Max_High',
                                     'Prv_7_High','Prv_6_High','Prv_5_High','Prv_4_High','Prv_3_High','Prv_2_High','Prv_1_High',
                                     '50_D_Min_Low','10_D_Min_Low',
                                     'Prv_7_Low','Prv_6_Low','Prv_5_Low','Prv_4_Low','Prv_3_Low','Prv_2_Low','Prv_1_Low',
                                     '100_D_Avg_Range',
                                     'Yr_H_Vol','Yr_L_Vol',
                                     '20_D_Avg_Vol','10_D_Avg_Vol','5_D_Avg_Vol',
                                     'Volume','Prev_Volume',
                                     'Prv_1_Open','Prv_2_Open','Prv_3_Open',
                                     'Prv_Yr_H_Price','Prv_Yr_L_Price','Prv_Yr_H_Vol','Prv_Yr_L_Vol',
                                     'Prv_8_High','Prv_8_Low']]
nifty_f_n_o_stocks_data.replace(np.nan,'',inplace = True)

In [18]:
nifty_f_n_o_stocks_data.shape

(146, 57)

In [19]:
# nifty_f_n_o_stocks_data.to_excel(excel_writer="nifty_f_n_o_stocks_data.xlsx")

In [20]:
# # to convert dataframe to list for uploading to google sheet
nifty_f_n_o_stocks_data = nifty_f_n_o_stocks_data.T.reset_index().T.values.tolist()

In [22]:
## writing list to google sheet
response_data = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        valueInputOption='USER_ENTERED',
        range='NSE_Futures',
        body=dict(
            majorDimension='ROWS',
            values=nifty_f_n_o_stocks_data)
        ).execute()

TypeError: Object of type int64 is not JSON serializable

In [23]:
## writing updated time to google sheet
import datetime
response_datetime = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        valueInputOption='USER_ENTERED',
        range='NSE_Futures!BF1',
        body={
            'values': [[datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')]]
        }
).execute()

## Nifty 500

In [ ]:
# nifty_f_n_o = pd.read_excel("stock_names.xlsx",header=0,sheet_name="NSE F&O")
nifty_500 = pd.read_excel("stock_names.xlsx",header=0,sheet_name="NSE 500")

In [ ]:
from tqdm import tqdm_notebook as tqdm
stock_data_nifty_500 = pd.DataFrame(columns = ['Stock Name', 'Open','High','Low','Close','Prev Close',
                                     'Yr_H_Price','Yr_L_Price','Trend',
                                     '200_D_Avg_Close','Prv_200_D_Avg_Close','120_D_Max_Close','50_D_Avg_Close',
                                     'Prv_50_D_Avg_Close','Prv_20_D_Avg_Close','20_D_Max_Close','20_D_Avg_Close',
                                     '5_D_Max_Close','5_D_Avg_Close','Prv_3_Close','Prv_2_Close',
                                     '50_D_Max_High','10_D_Max_High',
                                     'Prv_7_High','Prv_6_High','Prv_5_High','Prv_4_High','Prv_3_High','Prv_2_High','Prv_1_High',
                                     '50_D_Min_Low','10_D_Min_Low',
                                     'Prv_7_Low','Prv_6_Low','Prv_5_Low','Prv_4_Low','Prv_3_Low','Prv_2_Low','Prv_1_Low',
                                     '100_D_Avg_Range',
                                     'Yr_H_Vol','Yr_L_Vol',
                                     '20_D_Avg_Vol','10_D_Avg_Vol','5_D_Avg_Vol',
                                     'Volume','Prev_Volume',
                                     'Prv_1_Open','Prv_2_Open','Prv_3_Open',
                                     'Prv_Yr_H_Price','Prv_Yr_L_Price','Prv_Yr_H_Vol','Prv_Yr_L_Vol',
                                     'Prv_8_High','Prv_8_Low'])

stock_names = nifty_500['NSE 500 Stocks']

# from tqdm.notebook import tqdm
for stock in tqdm(stock_names):
    temp = get_stock_data(stock)
    temp = temp[['Stock Name', 'Open','High','Low','Close','Prev Close',
                                     'Yr_H_Price','Yr_L_Price','Trend',
                                     '200_D_Avg_Close','Prv_200_D_Avg_Close','120_D_Max_Close','50_D_Avg_Close',
                                     'Prv_50_D_Avg_Close','Prv_20_D_Avg_Close','20_D_Max_Close','20_D_Avg_Close',
                                     '5_D_Max_Close','5_D_Avg_Close','Prv_3_Close','Prv_2_Close',
                                     '50_D_Max_High','10_D_Max_High',
                                     'Prv_7_High','Prv_6_High','Prv_5_High','Prv_4_High','Prv_3_High','Prv_2_High','Prv_1_High',
                                     '50_D_Min_Low','10_D_Min_Low',
                                     'Prv_7_Low','Prv_6_Low','Prv_5_Low','Prv_4_Low','Prv_3_Low','Prv_2_Low','Prv_1_Low',
                                     '100_D_Avg_Range',
                                     'Yr_H_Vol','Yr_L_Vol',
                                     '20_D_Avg_Vol','10_D_Avg_Vol','5_D_Avg_Vol',
                                     'Volume','Prev_Volume',
                                     'Prv_1_Open','Prv_2_Open','Prv_3_Open',
                                     'Prv_Yr_H_Price','Prv_Yr_L_Price','Prv_Yr_H_Vol','Prv_Yr_L_Vol',
                                     'Prv_8_High','Prv_8_Low']]
    stock_data_nifty_500=stock_data_nifty_500.append(temp, ignore_index = True)

In [ ]:
nifty_500_stocks_data = pd.merge(stock_data_nifty_500, nifty_500, how='left', left_on=['Stock Name'], 
                                   right_on = [nifty_500.columns[0]])

In [ ]:
# # to convert dataframe to list for uploading to google sheet
nifty_500_stocks_data = nifty_500_stocks_data[['Stock Name', 'Sector', 'Open','High','Low','Close','Prev Close',
                                     'Yr_H_Price','Yr_L_Price','Trend',
                                     '200_D_Avg_Close','Prv_200_D_Avg_Close','120_D_Max_Close','50_D_Avg_Close',
                                     'Prv_50_D_Avg_Close','Prv_20_D_Avg_Close','20_D_Max_Close','20_D_Avg_Close',
                                     '5_D_Max_Close','5_D_Avg_Close','Prv_3_Close','Prv_2_Close',
                                     '50_D_Max_High','10_D_Max_High',
                                     'Prv_7_High','Prv_6_High','Prv_5_High','Prv_4_High','Prv_3_High','Prv_2_High','Prv_1_High',
                                     '50_D_Min_Low','10_D_Min_Low',
                                     'Prv_7_Low','Prv_6_Low','Prv_5_Low','Prv_4_Low','Prv_3_Low','Prv_2_Low','Prv_1_Low',
                                     '100_D_Avg_Range',
                                     'Yr_H_Vol','Yr_L_Vol',
                                     '20_D_Avg_Vol','10_D_Avg_Vol','5_D_Avg_Vol',
                                     'Volume','Prev_Volume',
                                     'Prv_1_Open','Prv_2_Open','Prv_3_Open',
                                     'Prv_Yr_H_Price','Prv_Yr_L_Price','Prv_Yr_H_Vol','Prv_Yr_L_Vol',
                                     'Prv_8_High','Prv_8_Low']]
nifty_500_stocks_data.replace(np.nan,'',inplace = True)

In [ ]:
# nifty_500_stocks_data[nifty_500_stocks_data['Stock Name']=="CHOLAFIN"]

In [ ]:
# # to convert dataframe to list for uploading to google sheet
nifty_500_stocks_data = nifty_500_stocks_data.T.reset_index().T.values.tolist()

In [ ]:
# nifty_500_stocks_data.to_excel(excel_writer="nifty_500_stocks_data.xlsx")

In [ ]:
## writing list to google sheet
response_data = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        valueInputOption='USER_ENTERED',
        range='NSE_500',
        body=dict(
            majorDimension='ROWS',
            values=nifty_500_stocks_data)
        ).execute()

In [ ]:
## writing updated time to google sheet
import datetime
response_datetime = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        valueInputOption='USER_ENTERED',
        range='NSE_500!BF1',
        body={
            'values': [[datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')]]
        }
).execute()